In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn. metrics import confusion_matrix, ConfusionMatrixDisplay , precision_score, recall_score, f1_score, r2_score, roc_curve, roc_auc_score, classification_report

2023-11-08 00:30:15.676628: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-08 00:30:16.296373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 00:30:17.752355: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 00:30:17.752410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 00:30:17.758583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
feature_vector = np.load("./dl_feature_c4.npy")
display(feature_vector.shape)

(16827, 130)

In [3]:
labels = np.load("./labels_v3.npy")
display(labels.shape)

(16827,)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(feature_vector,labels,test_size=0.3)

In [5]:
display(y_train.shape)

(11778,)

In [4]:
# fold = StratifiedKFold(n_splits=10)

In [5]:
# for train_index, test_index in fold.split(feature_vector, labels):
#     x_train, x_test, y_train, y_test = [feature_vector[i] for i in train_index], [feature_vector[i] for i in test_index], [labels[i] for i in train_index], [labels[i] for i in test_index]

In [6]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

In [12]:
# Input layer
inputs = layers.Input(shape=(130,))

# Hidden layers
x = layers.Dense(64, activation='relu')(inputs) 
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dense(8, activation='relu')(x) 

# Additional hidden layers
x = layers.Dense(16, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x) 
x = layers.Dense(64, activation='relu')(x)

# Classification layers  
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(4, activation='softmax')(x)

# Model
model = models.Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
369/369 [==============================] - 1s 2ms/step - loss: 1.3690 - accuracy: 0.3033
Epoch 2/10
369/369 [==============================] - 1s 2ms/step - loss: 1.3536 - accuracy: 0.3231
Epoch 3/10
369/369 [==============================] - 1s 2ms/step - loss: 1.3471 - accuracy: 0.3324
Epoch 4/10
369/369 [==============================] - 1s 2ms/step - loss: 1.3419 - accuracy: 0.3340
Epoch 5/10
369/369 [==============================] - 1s 2ms/step - loss: 1.3379 - accuracy: 0.3377
Epoch 6/10
369/369 [==============================] - 1s 1ms/step - loss: 1.3368 - accuracy: 0.3367
Epoch 7/10
369/369 [==============================] - 1s 1ms/step - loss: 1.3343 - accuracy: 0.3352
Epoch 8/10
369/369 [==============================] - 1s 1ms/step - loss: 1.3316 - accuracy: 0.3400
Epoch 9/10
369/369 [==============================] - 1s 1ms/step - loss: 1.3336 - accuracy: 0.3324
Epoch 10/10
369/369 [==============================] - 1s 1ms/step - loss: 1.3302 - accuracy: 0.3412

In [13]:
y_pred = model.predict(x_test)

158/158 [==============================] - 0s 905us/step


In [22]:
print(y_test)

[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [15]:
def plot_cnf(cnf_matrix):
  fig = px.imshow(cnf_matrix, 
                 color_continuous_scale='Blues')

  fig.update_layout(
      title="Confusion Matrix with Rest-case-labeled: 0, Zero-back-labeled: 1,One-back-labeled: 2, Two-back-labeled: 3",
      xaxis_title="Actual Labels",
      yaxis_title="Predicted Labels",
      width= 700,
      height=700,
  )

  fig.update_layout(
      font=dict(
          size=12
      ),
      xaxis = dict(
          tick0=0,
          dtick=1
      ),
      yaxis = dict(
          tick0=0,
          dtick=1
      )
  )

  for i in range(len(cnf_matrix)):
      for j in range(len(cnf_matrix)):
          
          if (cnf_matrix[i,j] >= 220):
              color = 'white'
          else: 
              color = 'black'

          fig.add_annotation(text=str(cnf_matrix[i,j]), 
                            x=j, 
                            y=i, 
                            showarrow=False,
                            font=dict(
                                color=color,
                                size=24,
                            ) 
                            )

  fig.show()

In [16]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
plot_cnf(confusion_mtx)

In [25]:
def plot_roc(tpr, fpr):
  fig = px.line(x=fpr, 
              y=tpr,
              )
  fig.add_scatter(x=[0,1], 
                  y=[0,1],
                  line=dict(color='navy', dash='dash'),
                  name="Guessing"
                  )

  fig.update_layout(
      title='ROC Curve',
      xaxis_title='False Positive Rate',
      yaxis_title='True Positive Rate',
      width=700,
      height=500,
  )

  fig.show()

In [33]:
# Performance metrics calculation function

def metrics(model, labels = labels, epoch=10):
  # Using average of cross val score for accuracy
  # score = cross_val_score(model, 
  #                                 data, 
  #                                 labels,
  #                                 cv=10,
  #                               )
  
  # cross_val_acc = np.average(score)

  # Training the model on x_train and y_train
  model.fit(x_train, y_train, epochs=epoch)

  # Getting the class-label predictions and class-label prediction probabilities from the trained model
  model_predictions = model.predict(x_test)         # model_predictions.shape = [len(x-test)], (label(ith epoch))
  # y_pred_prob = model.predict_proba(x_test)         # y_pred_prob.shape = [len(x-test), 2], (prob(label0), prob(label1))

  # Building the Classification Report using the predictions as a dataframe without the accuracy column
  # classif_report = pd.DataFrame(classification_report(y_true=y_test, 
  #                                                     y_pred=model_predictions, 
  #                                                     output_dict=True,
  #                                                     zero_division=0,
  #                                                     )
  #                               ).drop(labels="accuracy", axis=1).T.round(2)
  
  # Building the Confusion Matrix using the predicted class labels
  cnf_matrix = confusion_matrix(y_true=y_test, y_pred=model_predictions)
  
  # if len(np.unique(labels)) == 2:
  #   # Getting the FalsePositiveRate and TruePositveRates from plotting the ROC curve
  #   fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])

  # if len(np.unique(labels)) == 2:
  #   # Calculating the Area under the ROC curve ie, AUC using class label prediction probabilities
  #   auc = roc_auc_score(y_test, y_pred_prob[:,1])

  # else:
  #   micro_roc_auc_ovr = roc_auc_score(
  #                                     y_test,
  #                                     y_pred_prob,
  #                                     multi_class="ovr",
  #                                     average="micro",
  #                                     )
    
  #   macro_roc_auc_ovr = roc_auc_score(
  #                                     y_test,
  #                                     y_pred_prob,
  #                                     multi_class="ovr",
  #                                     average="macro",
  #                                     )
    
  #   macro_roc_auc_ovo = roc_auc_score(
  #                                     y_test,
  #                                     y_pred_prob,
  #                                     multi_class="ovo",
  #                                     average="macro",
  #                                     )
    
  # print(f"Cross-val-mean-Accuracy: {cross_val_acc:.4f}\n")

  # display(classif_report)

  plot_cnf(cnf_matrix=cnf_matrix)

  # if len(np.unique(labels)) == 2:
  #   plot_roc(tpr=tpr,
  #           fpr=fpr,
  #           )
  
  # if len(np.unique(labels)) == 2:
  #   print(f"Area Under the ROC Curve (AUC): {auc:.4f}")
  
  # else:
  #   print(f"\nMicro-averaged One-vs-Rest ROC AUC score:{micro_roc_auc_ovr:.2f}")
  #   print(f"\nMacro-averaged One-vs-Rest ROC AUC score:{macro_roc_auc_ovr:.2f}")
  #   print(f"\nMacro-averaged One-vs-One ROC AUC score:{macro_roc_auc_ovo:.2f}")

In [34]:
metrics(model=model, epoch=3)

Epoch 1/3
369/369 [==============================] - 1s 2ms/step - loss: 1.3144 - accuracy: 0.3573
Epoch 2/3
369/369 [==============================] - 1s 2ms/step - loss: 1.3136 - accuracy: 0.3588
Epoch 3/3
158/158 [==============================] - 0s 950us/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets